In [12]:
# 对训练的网络进行测试
import numpy as np
import torch
from torchvision import transforms
from dnnbrain.dnn.io import PicDataset, DataLoader
from cnnface.dnn.vgg_identity_recons import Vgg_identity
from dnnbrain.dnn.models import dnn_test_model

# load noise image
imgcsv_path =  'D:\cnnface\Emotion_analysis/happy_sad_test_400.csv'
transform = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor()])
PicSet = PicDataset(imgcsv_path, transform)
Picloader = DataLoader(PicSet, batch_size=32,shuffle=False)
# load model
vggid = Vgg_identity()
vggid.load_state_dict(torch.load('F:/Code/pretrained_model/vgg_emotion_CrossEntro.pth'))

# Get Classification result of vgg
label, expect_label, accuracy = dnn_test_model(Picloader, vggid)
print(accuracy)

Now loading batch 1


RuntimeError: CUDA out of memory. Tried to allocate 392.00 MiB (GPU 0; 4.00 GiB total capacity; 1.02 GiB already allocated; 0 bytes free; 383.48 MiB cached)

In [ ]:
# extract dnn_activation
from torchvision import transforms
from torch.utils.data import DataLoader
from dnnbrain.dnn import analyzer
from dnnbrain.dnn import io as dnn_io

stim = 'D:\cnnface\Identity_analysis/identity2_train.csv'
netloader = dnn_io.NetLoader('vggface')
imgcropsize = netloader.img_size
transform = transforms.Compose([transforms.Resize(imgcropsize),
                                    transforms.ToTensor()])  
picdataset = dnn_io.PicDataset(stim, transform=transform)
picdataloader = DataLoader(picdataset, batch_size=8, shuffle=False)
dnn_act = analyzer.dnn_activation(picdataloader, 'vggface', 'fc3')

In [15]:
import numpy as np
from dnnbrain.dnn import io
import os
from PIL import Image 
import sys
sys.path.append("..")
from cnnface.stimulus.Image_processing import average_img

csv_file = 'D:\cnnface\Emotion_analysis/noiseface.csv'
index_f = np.loadtxt('D:\cnnface\Emotion_analysis\CI_analysis/label_happy.txt').astype('int64')
index_m = np.loadtxt('D:\cnnface\Emotion_analysis\CI_analysis/label_sad.txt').astype('int64')

savepath_aver_f = 'D:\cnnface\Emotion_analysis\CI_analysis\ci/aver_happy_noise.jpg'
savepath_aver_m = 'D:\cnnface\Emotion_analysis\CI_analysis\ci/aver_sad_noise.jpg'
savepath_CI = 'D:\cnnface\Emotion_analysis\CI_analysis\ci/ci_noise.jpg'

#Get pictures path from stim_csv and label txt.
picset= io.PicDataset(csv_file)
pre_picpath = picset.picpath
sub_picpath = picset.picname

f_picpath = [os.path.join(pre_picpath,sub_picpath[i]) for i in index_f]
m_picpath = [os.path.join(pre_picpath,sub_picpath[i]) for i in index_m]

#average all of the pictures as their label.
aver_img_f = average_img(f_picpath)
aver_img_m = average_img(m_picpath)

#calculate the CI and superimpose image and subtract image.
ci = aver_img_f.astype('int64') - aver_img_m.astype('int64')
#ci = ci - ci.min()
ci = ci * 5
print(ci.max(),ci.min())

# (Image.fromarray(aver_img_f.astype('uint8'))).save(savepath_aver_f,quality=95)
# (Image.fromarray(aver_img_m.astype('uint8'))).save(savepath_aver_m,quality=95)
# (Image.fromarray(ci.astype('uint8'))).save(savepath_CI,quality=95)

30 -20


In [2]:
# savepath_CI = 'D:\cnnface\Emotion_analysis\CI_analysis\ci/ci_noise_nor.jpg'
# nor = lambda x:(x-x.min())/(x.max() - x.min()) 
# ci  = nor(ci) * 255
# (Image.fromarray(ci.astype('uint8'))).save(savepath_CI,quality=95)

In [34]:
base_face = np.array(Image.open(r'D:\cnnface\Emotion_analysis\face_template/baseface.jpg')).astype('int64')

ci =  np.array(Image.open('D:\cnnface\Emotion_analysis\CI_analysis\ci/ci_noise.jpg')).astype('int64')

bf_add = base_face + ci 
bf_sub = base_face - ci 

bf_add[bf_add>255] = 255
bf_sub[bf_sub<0] = 0

# nor = lambda x:(x-x.min())/(x.max() - x.min()) 
# bf_add_x = nor(bf_add) *255
# bf_sub_x = nor(bf_sub) *255
(Image.fromarray(bf_add.astype('uint8'))).save('D:\cnnface\Emotion_analysis\CI_analysis\ci/bf_add.jpg',quality=95)
(Image.fromarray(bf_sub.astype('uint8'))).save('D:\cnnface\Emotion_analysis\CI_analysis\ci/bf_sub.jpg',quality=95)

-20


In [ ]:
import numpy as np
from PIL import Image

usecols = list(range(513))
usecols.remove(0)
sinusoid = np.loadtxt('D:/cnnface/rcrci/sinusoid/cycle/sinuoid_4.csv',delimiter=',',skiprows=1,usecols=usecols)
sinusoid = ((sinusoid - sinusoid.min())/(sinusoid.max() - sinusoid.min()))*255
img_sinu = Image.fromarray(sinusoid)
img_sinu.show()


In [ ]:
import sys
sys.path.append("..")
from cnnface.stimulus import Image_processing

CI_ori_p = 'D:/cnnface/female_male_test_51_addnoise\Face_template/classification_noise/CI_ori.jpg'
CI_ori_5 = 'D:/cnnface/female_male_test_51_addnoise\Face_template/classification_noise/CI_ori_5.jpg'
CI_ori_20 = 'D:/cnnface/female_male_test_51_addnoise\Face_template/classification_noise/CI_ori_20.jpg'
CI_sinusoid_nor = 'D:/cnnface/female_male_test_51_addnoise\Face_template/classification_noise/CI_sinusoid_nor.jpg'
CI_sinusoid5_nor = 'D:/cnnface/female_male_test_51_addnoise\Face_template/classification_noise/CI_sinusoid5_nor.jpg'

# Image_processing.image_freq_hist_plot(CI_ori_p)
# Image_processing.image_freq_hist_plot(CI_ori_5)
# Image_processing.image_freq_hist_plot(CI_ori_20)
# Image_processing.image_freq_hist_plot(CI_sinusoid_nor)
# Image_processing.image_freq_hist_plot(CI_sinusoid5_nor)


Image_processing.img_zmap(CI_ori_p)
Image_processing.img_zmap(CI_ori_5)
Image_processing.img_zmap(CI_ori_20)
Image_processing.img_zmap(CI_sinusoid_nor)

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as pg

CI_sinusoid_nor = 'D:/cnnface/female_male_test_51_addnoise\Face_template/classification_noise/CI_sinusoid_nor.jpg'
CI = Image.open(CI_sinusoid_nor)
CI_arr = np.array(CI)

ci_map = plt.imshow(CI_arr,cmap = 'jet',interpolation='nearest')
save_path  ='D:/cnnface/female_male_test_51_addnoise/Face_template/classification_noise/CI_jet.jpg'
pg.imsave(save_path,CI_arr,cmap = 'jet',format='jpg',dpi=1000)



In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image

import matplotlib.pyplot as plt

#read image path from csv.file
csv_file = 'D:/cnnface/female_male_test_51_addnoise/rcicr_sinusoid_noise.csv'
pd = pd.read_csv(csv_file,skiprows=1)
stimID = np.array(pd['stimID'])
with open(csv_file,'r') as f:
    picpath = f.readline().rstrip()
    
#read base img 
base_img = Image.open('D:/cnnface/female_male_test_51_addnoise/Face_template/face_template/frame054_gray_512.jpg')
baseimg_arr = np.array(base_img).astype('int32')
img_min = []
img_mean = []
img_max = []

#read image from array
#calculate the min mean max of singal pictures
for x,i in enumerate(stimID):
    picimg = Image.open(os.path.join(picpath, i))
    picimg_arr = np.array(picimg).astype('int32')
    contrast = picimg_arr - baseimg_arr
    # - base image to get the pixel (positive and negative)
    con_min = contrast.min()
    con_mean = contrast.mean()
    con_max = contrast.max()
    
    if x <5:
        plt.hist(contrast)
        plt.show()
    img_min.append(con_min)
    img_mean.append(con_mean)
    img_max.append(con_max)
#sum and plot the hist of min ,mean,max of all pictures

plt.hist(img_min)
plt.show()
plt.hist(img_mean)
plt.show()
plt.hist(img_max)
plt.show()

In [ ]:
#load RData file to python
import rpy2.robjects as robjects
import numpy as np
robjects.r['load']("D:/cnnface/female_male_test_51_addnoise/Face_template/meta_data/pythonRead/sinusoid1000_patches.RData")

x = robjects.r['patches']
print(np.array(x))

In [ ]:
#convert .RData to npy file
import rpy2.robjects as robjects
import numpy as np

robjects.r['load']("D:/cnnface/female_male_test_51_addnoise/Face_template/meta_data/rcic_seed_1_time_9月_18_2019_10_07.Rdata")
params = robjects.r['stimuli_params'][0]

params_1000 = np.array(params)

robjects.r['load']("D:/cnnface/female_male_test_51_addnoise/Face_template/meta_data/rcic_seed_1_time_9月_18_2019_10_30.Rdata")
params = robjects.r['stimuli_params'][0]

params_2000 = np.array(params)

robjects.r['load']("D:/cnnface/female_male_test_51_addnoise/Face_template/meta_data/rcic_seed_1_time_9月_18_2019_10_45.Rdata")
params = robjects.r['stimuli_params'][0]

params_3000 = np.array(params)

robjects.r['load']("D:/cnnface/female_male_test_51_addnoise/Face_template/meta_data/rcic_seed_1_time_9月_18_2019_11_02.Rdata")
params = robjects.r['stimuli_params'][0]

params_4000 = np.array(params)

robjects.r['load']("D:/cnnface/female_male_test_51_addnoise/Face_template/meta_data/rcic_seed_1_time_9月_18_2019_11_22.Rdata")
params = robjects.r['stimuli_params'][0]

params_5000 = np.array(params)

p = np.concatenate((params_1000,params_2000),axis=0)
p = np.concatenate((p,params_3000),axis=0)
p = np.concatenate((p,params_4000),axis=0)
p = np.concatenate((p,params_5000),axis=0)

np.save('D:/cnnface/female_male_test_51_addnoise/Face_template/meta_data/params_5000',p)



In [ ]:
#convert .RData to npy file
import rpy2.robjects as robjects
import numpy as np

robjects.r['load']("D:/cnnface/female_male_test_51_addnoise/Face_template/meta_data/rcic_seed_1_time_9月_18_2019_10_07.Rdata")
p = robjects.r['p']
print(p.names)
patches = p[0]
patchidx = p[1]

patches = np.array(patches)
patchidx = np.array(patchidx)

np.save('D:/cnnface/female_male_test_51_addnoise/Face_template/meta_data/patches.npy',patches)
np.save('D:/cnnface/female_male_test_51_addnoise/Face_template/meta_data/patchidx.npy',patchidx)


